In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
customer_nodes = pd.read_csv(r"C:\Users\sathw\batch 261\case studies\case study-4\customer_nodes.csv")
customer_transactions = pd.read_csv(r"C:\Users\sathw\batch 261\case studies\case study-4\customer_transactions.csv")
regions = pd.read_csv(r"C:\Users\sathw\batch 261\case studies\case study-4\regions.csv")

In [7]:
customer_nodes

,Unnamed: 0,customer_id,region_id,node_id,start_date,end_date
0,0,1,3,4,2020-01-02,2020-01-03
1,1,2,3,5,2020-01-03,2020-01-17
2,2,3,5,4,2020-01-27,2020-02-18
3,3,4,5,4,2020-01-07,2020-01-19
4,4,5,3,3,2020-01-15,2020-01-23
...,...,...,...,...,...,...
3495,3495,496,3,4,2020-02-25,9999-12-31
3496,3496,497,5,4,2020-05-27,9999-12-31
3497,3497,498,1,2,2020-04-05,9999-12-31
3498,3498,499,5,1,2020-02-03,9999-12-31


In [8]:
customer_transactions

,Unnamed: 0,customer_id,txn_date,txn_type,txn_amount
0,0,429,2020-01-21,deposit,82
1,1,155,2020-01-10,deposit,712
2,2,398,2020-01-01,deposit,196
3,3,255,2020-01-14,deposit,563
4,4,185,2020-01-29,deposit,626
...,...,...,...,...,...
5863,5863,189,2020-02-03,withdrawal,870
5864,5864,189,2020-03-22,purchase,718
5865,5865,189,2020-02-06,purchase,393
5866,5866,189,2020-01-22,deposit,302


In [9]:
regions

,Unnamed: 0,region_id,region_name
0,0,1,Australia
1,1,2,America
2,2,3,Africa
3,3,4,Asia
4,4,5,Europe


### Customer Nodes Exploration
#### 1) How many unique nodes are there on the Data Bank system?

In [13]:
# to find the no.of unique nodes in DBS system
# using 'nunique() function of the 'node_id' column in 'customer_nodes' df
unique_nodes_count = customer_nodes['node_id'].nunique()
print("Number of unique nodes in the Data Bank system:", unique_nodes_count)

Number of unique nodes in the Data Bank system: 5


#### 2) What is the number of nodes per region?

In [22]:
# To find the number of nodes per region, you can use the groupby() function along with nunique() on the 'region_id' and 'node_id' columns in the 'customer_nodes' DataFrame.
nodes_per_region = customer_nodes.groupby('region_id')['node_id'].nunique().reset_index()

# This code will group the data by 'region_id' and calculate the number of unique nodes within each region.
# The resulting DF will display the 'region_id' and the corresponding number of nodes for each region.
nodes_per_region.columns = ['region_id', 'number_of_nodes']

print("Number of nodes per region: ", nodes_per_region)

Number of nodes per region:     region_id  number_of_nodes
0          1                5
1          2                5
2          3                5
3          4                5
4          5                5


#### 3) How many customers are allocated to each region?

In [23]:
# no.of customers allocated to each region, by using 'groupby' function along with 'count' on 'region_id' column in 'customer_nodes' df.
customers_per_region = customer_nodes.groupby('region_id')['customer_id'].count().reset_index()
# it groups the data by'region_id' and count the number of customers in each region.
customers_per_region.columns = ['region_id', 'number_of_customers']
print("Number of customers allocated to each region:")
print(customers_per_region)

Number of customers allocated to each region:
   region_id  number_of_customers
0          1                  770
1          2                  735
2          3                  714
3          4                  665
4          5                  616


#### 4) How many days on average are customers reallocated to a different node?

In [29]:
# Convert 'start_date' and 'end_date' columns to datetime objects
customer_nodes['start_date'] = pd.to_datetime(customer_nodes['start_date'])
customer_nodes['end_date'] = pd.to_datetime(customer_nodes['end_date'])

# Calculate the duration of each reallocation
customer_nodes['reallocation_days'] = (customer_nodes['end_date'] - customer_nodes['start_date']).dt.days

# Calculate the average number of days customers are reallocated
average_reallocation_days = customer_nodes['reallocation_days'].mean()

print(f"The average number of days customers are reallocated to a different node is: {average_reallocation_days:.2f} days")

The average number of days customers are reallocated to a different node is: 14.63 days


#### 5) What is the median, 80th and 95th percentile for this same reallocation days metric for each region?

In [31]:
# Calculate reallocation days for each customer
customer_nodes['reallocation_days'] = (customer_nodes['end_date'] - customer_nodes['start_date']).dt.days

# Group by region and calculate the metrics
region_metrics = customer_nodes.groupby('region_id')['reallocation_days'].agg(['median', lambda x: np.percentile(x, 80), lambda x: np.percentile(x, 95)])

# Renaming columns for better readability
region_metrics.columns = ['median_reallocation_days', '80th_percentile_reallocation_days', '95th_percentile_reallocation_days']
region_metrics

,median_reallocation_days,80th_percentile_reallocation_days,95th_percentile_reallocation_days
region_id,,,
1,15.0,NaN,NaN
2,15.0,NaN,NaN
3,15.0,NaN,NaN
4,15.0,NaN,NaN
5,15.0,NaN,NaN


### Customer Transactions
#### 1) What is the unique count and total amount for each transaction type?

In [38]:
# Group by transaction type and calculate unique count and total amount
transaction_summary = customer_transactions.groupby('txn_type').agg({'customer_id': 'nunique', 'txn_amount': 'sum'})

# Rename columns for better readability
transaction_summary.columns = ['unique_customer_count', 'total_amount']
transaction_summary

,unique_customer_count,total_amount
txn_type,,
deposit,500,1359168
purchase,448,806537
withdrawal,439,793003


#### 2) What is the average total historical deposit counts and amounts for all customers?

In [40]:
# Filtering deposit transactions
deposit_transactions = customer_transactions[customer_transactions['txn_type'] == 'deposit']

# making Group by customer and calculate total deposit counts and amounts
deposit_summary = deposit_transactions.groupby('customer_id').agg({'txn_amount': ['count', 'sum']})

average_deposit_counts = deposit_summary['txn_amount']['count'].mean()
average_deposit_amounts = deposit_summary['txn_amount']['sum'].mean()

print("Average Total Historical Deposit Counts:", average_deposit_counts)
print("Average Total Historical Deposit Amounts:", average_deposit_amounts)

Average Total Historical Deposit Counts: 5.342
Average Total Historical Deposit Amounts: 2718.336


#### 3) For each month - how many Data Bank customers make more than 1 deposit and either 1 purchase or 1 withdrawal in a single month?

In [41]:
# Converting txn_date to datetime
customer_transactions['txn_date'] = pd.to_datetime(customer_transactions['txn_date'])

# Extracting month and year from txn_date
customer_transactions['month'] = customer_transactions['txn_date'].dt.to_period('M')

# Filtering relevant transactions
relevant_transactions = customer_transactions[(customer_transactions['txn_type'].isin(['deposit', 'purchase', 'withdrawal'])) & (customer_transactions['txn_amount'] > 0)]

# making Group by customer and month, then count unique transaction types
monthly_activity = relevant_transactions.groupby(['customer_id', 'month'])['txn_type'].nunique()

# Counting customers meeting the criteria for each month
active_customers_per_month = monthly_activity[(monthly_activity > 1)].groupby('month').count()

print(active_customers_per_month)

month
2020-01    295
2020-02    342
2020-03    364
2020-04    156
Freq: M, Name: txn_type, dtype: int64


#### 4) What is the closing balance for each customer at the end of the month?

In [43]:
# Convert txn_date to datetime
customer_transactions['txn_date'] = pd.to_datetime(customer_transactions['txn_date'])

# Extract month and year from txn_date
customer_transactions['month'] = customer_transactions['txn_date'].dt.to_period('M')

# Calculate closing balance for each customer at the end of the month
closing_balance = customer_transactions.groupby(['customer_id', 'month'])['txn_amount'].sum()
closing_balance

customer_id  month  
1            2020-01     312
             2020-03    1600
2            2020-01     549
             2020-03      61
3            2020-01     144
                        ... 
499          2020-02    3007
             2020-03    2324
500          2020-01    3566
             2020-02    1485
             2020-03    4792
Name: txn_amount, Length: 1720, dtype: int64